In [1]:
import torch
from torch import nn
from torchvision.transforms import ToTensor
import pandas as pd

In [2]:
from LSTM_model import NeuralNetwork
from run_model import compile_model,predict_model

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [4]:
# 손실 함수를 초기화합니다.
loss_fn = nn.CrossEntropyLoss()

In [5]:
df_data = pd.read_csv("bibim_learndata.csv")
df_data.columns

Index(['textLevel', 'textContent'], dtype='object')

In [8]:
# from konlpy.tag import Okt
# tokenizer = Okt()

from tensorflow.keras.preprocessing.text import Tokenizer
vocab_size = 10000
oov_tok ='<oov>'
tokenizer = Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer

In [9]:
tokenizer.fit_on_texts(df_data['textContent'])
word_to_index = tokenizer.word_index
sorted(word_to_index)[:10]

["'",
 "'ㅍㄷ비빔면'보다",
 "'겨울맞이",
 "'배홍동이",
 "'윈터에디션'으로",
 "'입니다",
 "'자주구매'",
 '0',
 '000원이상',
 '00에']

In [10]:
train_sequences = tokenizer.texts_to_sequences([str(text) for text in df_data['textContent']])

In [11]:
max_length = 500
padding_type = 'post' # 뒤에 채우기

In [12]:
from keras.preprocessing.sequence import pad_sequences
feature_train_sp = pad_sequences(train_sequences,padding=padding_type,maxlen=max_length)

In [15]:
len(feature_train_sp[0])

500

In [16]:
class_num = len(df_data['textLevel'].unique())
data_len = len(feature_train_sp[0])
input_data = torch.rand(1, 28, 28) # 여기가 데이터 들어가는 곳

In [18]:
model = compile_model(data_len,class_num)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")